# 1. Imports of necessary libraries

In [13]:
import keras #High level interface of TensorFlow
from keras.models import Sequential
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.utils.np_utils import to_categorical 
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
from keras.optimizers import RMSprop
import tensorflowjs as tfjs
import pandas as pd
from sklearn.model_selection import train_test_split

# 2.Data importing& preprocessing

## 2.1 Importing and entry processing

In [22]:
train_set = pd.read_csv("../data/train.csv") # raw data

# DV
Y_train = train_set["label"]
# IV
X_train = train_set.drop(labels = ["label"], axis = 1)
#X_train

## 2.2Normalization to [0-1]  range

In [23]:
X_train = X_train / 255


X_train = X_train.values.reshape(-1,28,28,1)

Y_train = to_categorical(Y_train, num_classes = 10)

## 2.3Data spliting into train & valid set 

In [24]:
X_train, X_valid , Y_train , Y_valid = train_test_split(X_train, Y_train, test_size = 0.2, random_state = 42)

## 2.4 Augmentation

In [25]:
igd = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=10,
    zoom_range = 0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)

igd.fit(X_train)

# 3. Building the architecture of Keras model & Compilation

## 3.1 Architecture

In [26]:
model = Sequential(
        [
            Conv2D(32, (3,3),activation = "relu",padding = 'Same',input_shape = (28,28,1)),
            Conv2D(32, (3,3),activation = "relu",padding = 'Same'),
            MaxPool2D(pool_size = (2, 2)),
            Flatten(),
            Dropout(0.25),
            Dense(units = 64, activation = "relu"),
            Dense(units = 10, activation = "sigmoid")  
        ]
)

## 3.2 Compilation

In [27]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

# 4. Training

In [28]:
model.fit_generator(
    igd.flow(X_train, Y_train, batch_size = 32),
    epochs = 50,
    steps_per_epoch = X_train.shape[0] // 32,
    validation_data = (X_valid, Y_valid)
    )

Epoch 1/50
1050/1050 [==============================] - 118s 112ms/step - loss: 0.4212 - acc: 0.8664 - val_loss: 0.0961 - val_acc: 0.9707
Epoch 2/50
1050/1050 [==============================] - 117s 111ms/step - loss: 0.1588 - acc: 0.9510 - val_loss: 0.0797 - val_acc: 0.9769
Epoch 3/50
1050/1050 [==============================] - 117s 111ms/step - loss: 0.1216 - acc: 0.9629 - val_loss: 0.0610 - val_acc: 0.9819
Epoch 4/50
1050/1050 [==============================] - 117s 111ms/step - loss: 0.1028 - acc: 0.9686 - val_loss: 0.0424 - val_acc: 0.9874
Epoch 5/50
1050/1050 [==============================] - 118s 112ms/step - loss: 0.0898 - acc: 0.9728 - val_loss: 0.0479 - val_acc: 0.9862
Epoch 6/50
1050/1050 [==============================] - 116s 111ms/step - loss: 0.0824 - acc: 0.9744 - val_loss: 0.0407 - val_acc: 0.9873
Epoch 7/50
1050/1050 [==============================] - 121s 115ms/step - loss: 0.0732 - acc: 0.9771 - val_loss: 0.0423 - val_acc: 0.9877
Epoch 8/50
1050/1050 [============

# Saving a model

In [21]:
tfjs.converters.save_keras_model(model, '../model')